In [5]:
# Importing required modules
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import seaborn as sns
from scipy import optimize
import pymc3 as pm
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow import keras
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from pickle import dump, load
from sklearn.metrics import confusion_matrix, classification_report
from numpy.random import seed
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
# Bayesian
import arviz as az
import graphviz

/opt/conda/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [11]:
crime = pd.read_stata('crime.dta')

In [12]:
comp = pd.read_csv("county_complete.csv")

In [13]:
crime.head()

,STUDYNO,EDITION,PART,IDNO,FIPS_ST,FIPS_CTY,CPOPARST,AG_ARRST,JURFLAG,COVIND,...,OFAGFAM,DUI,LIQUOR,DRUNK,DISORDR,VAGRANT,ALLOTHR,SUSPICN,CURFEW,RUNAWAY
0,This variable is no longer in use,"2nd ed, 2014","Arrests, All Ages",1,1,1,56219,3,Multiple counties,98.4525,...,26,182,107,69,17,2,1172,0,33,4
1,This variable is no longer in use,"2nd ed, 2014","Arrests, All Ages",2,1,3,182265,13,One cnty,100.0000,...,25,1285,485,855,146,22,3310,0,0,5
2,This variable is no longer in use,"2nd ed, 2014","Arrests, All Ages",3,1,5,27457,6,One cnty,98.9839,...,0,43,26,26,19,0,287,0,0,0
3,This variable is no longer in use,"2nd ed, 2014","Arrests, All Ages",4,1,7,22866,5,Multiple counties,100.0000,...,19,58,16,6,21,0,336,0,0,0
4,This variable is no longer in use,"2nd ed, 2014","Arrests, All Ages",5,1,9,57322,6,One cnty,99.2813,...,9,178,46,42,17,0,1048,0,0,1


In [14]:
comp.head()

,fips,state,name,pop2000,pop2010,pop2011,pop2012,pop2013,pop2014,pop2015,...,poverty_under_18_2019,two_plus_races_2019,unemployment_rate_2019,uninsured_2019,uninsured_65_and_older_2019,uninsured_under_19_2019,uninsured_under_6_2019,veterans_2019,white_2019,white_not_hispanic_2019
0,1001,Alabama,Autauga County,43671.0,54571,55199.0,54927.0,54695.0,54864.0,54838.0,...,23.2,2.2,3.5,7.1,0.0,1.7,1.7,12.6,76.8,74.6
1,1003,Alabama,Baldwin County,140415.0,182265,186534.0,190048.0,194736.0,199064.0,202863.0,...,13.4,1.7,4.0,8.9,0.3,3.8,2.2,11.8,86.2,83.1
2,1005,Alabama,Barbour County,29038.0,27457,27351.0,27175.0,26947.0,26749.0,26264.0,...,50.1,1.2,9.4,11.3,0.3,3.3,3.4,6.6,46.8,45.8
3,1007,Alabama,Bibb County,20826.0,22915,22745.0,22658.0,22503.0,22533.0,22561.0,...,NaN,0.6,7.0,10.7,0.0,2.0,4.5,8.0,76.8,74.5
4,1009,Alabama,Blount County,51024.0,57322,57562.0,57595.0,57623.0,57546.0,57590.0,...,18.4,1.6,3.1,10.8,0.2,5.9,6.1,7.7,95.5,86.9


In [20]:
crime['FIPS_ST'] = crime['FIPS_ST'].astype(str)

In [21]:
crime['FIPS_ST'] = crime['FIPS_ST'].str.zfill(2)

In [23]:
crime.tail()

,STUDYNO,EDITION,PART,IDNO,FIPS_ST,FIPS_CTY,CPOPARST,AG_ARRST,JURFLAG,COVIND,...,OFAGFAM,DUI,LIQUOR,DRUNK,DISORDR,VAGRANT,ALLOTHR,SUSPICN,CURFEW,RUNAWAY
3172,This variable is no longer in use,"2nd ed, 2014","Arrests, All Ages",3173,56,39,21294,2,One cnty,100.0000,...,4,171,45,149,12,0,194,0,0,1
3173,This variable is no longer in use,"2nd ed, 2014","Arrests, All Ages",3174,56,41,21118,3,One cnty,89.9848,...,9,179,107,81,30,1,286,3,88,12
3174,This variable is no longer in use,"2nd ed, 2014","Arrests, All Ages",3175,56,43,8533,2,One cnty,100.0000,...,4,69,61,11,11,0,41,0,0,0
3175,This variable is no longer in use,"2nd ed, 2014","Arrests, All Ages",3176,56,45,7208,3,One cnty,84.7392,...,1,35,15,10,23,0,29,0,1,12
3176,This variable is no longer in use,"2nd ed, 2014","Arrests, All Ages",3177,56,777,0,1,One cnty,100.0000,...,0,0,0,0,0,0,0,0,0,0


In [24]:
crime['FIPS_CTY'] = crime['FIPS_CTY'].astype(str)

In [25]:
crime['FIPS_CTY'] = crime['FIPS_CTY'].str.zfill(3)

In [33]:
fips_list = pd.unique(crime["fips"])
print(fips_list)

['01001' '01003' '01005' ... '56043' '56045' '56777']


In [28]:
crime["fips"] = crime["FIPS_ST"] + crime["FIPS_CTY"]

In [38]:
crime.head()

,STUDYNO,EDITION,PART,IDNO,FIPS_ST,FIPS_CTY,CPOPARST,AG_ARRST,JURFLAG,COVIND,...,DUI,LIQUOR,DRUNK,DISORDR,VAGRANT,ALLOTHR,SUSPICN,CURFEW,RUNAWAY,fips
0,This variable is no longer in use,"2nd ed, 2014","Arrests, All Ages",1,01,001,56219,3,Multiple counties,98.4525,...,182,107,69,17,2,1172,0,33,4,01001
1,This variable is no longer in use,"2nd ed, 2014","Arrests, All Ages",2,01,003,182265,13,One cnty,100.0000,...,1285,485,855,146,22,3310,0,0,5,01003
2,This variable is no longer in use,"2nd ed, 2014","Arrests, All Ages",3,01,005,27457,6,One cnty,98.9839,...,43,26,26,19,0,287,0,0,0,01005
3,This variable is no longer in use,"2nd ed, 2014","Arrests, All Ages",4,01,007,22866,5,Multiple counties,100.0000,...,58,16,6,21,0,336,0,0,0,01007
4,This variable is no longer in use,"2nd ed, 2014","Arrests, All Ages",5,01,009,57322,6,One cnty,99.2813,...,178,46,42,17,0,1048,0,0,1,01009


In [34]:
comp['fips'] = comp['fips'].astype(str)

In [35]:
comp['fips'] = comp['fips'].str.zfill(5)

In [36]:
comp.head()

,fips,state,name,pop2000,pop2010,pop2011,pop2012,pop2013,pop2014,pop2015,...,poverty_under_18_2019,two_plus_races_2019,unemployment_rate_2019,uninsured_2019,uninsured_65_and_older_2019,uninsured_under_19_2019,uninsured_under_6_2019,veterans_2019,white_2019,white_not_hispanic_2019
0,01001,Alabama,Autauga County,43671.0,54571,55199.0,54927.0,54695.0,54864.0,54838.0,...,23.2,2.2,3.5,7.1,0.0,1.7,1.7,12.6,76.8,74.6
1,01003,Alabama,Baldwin County,140415.0,182265,186534.0,190048.0,194736.0,199064.0,202863.0,...,13.4,1.7,4.0,8.9,0.3,3.8,2.2,11.8,86.2,83.1
2,01005,Alabama,Barbour County,29038.0,27457,27351.0,27175.0,26947.0,26749.0,26264.0,...,50.1,1.2,9.4,11.3,0.3,3.3,3.4,6.6,46.8,45.8
3,01007,Alabama,Bibb County,20826.0,22915,22745.0,22658.0,22503.0,22533.0,22561.0,...,NaN,0.6,7.0,10.7,0.0,2.0,4.5,8.0,76.8,74.5
4,01009,Alabama,Blount County,51024.0,57322,57562.0,57595.0,57623.0,57546.0,57590.0,...,18.4,1.6,3.1,10.8,0.2,5.9,6.1,7.7,95.5,86.9


In [37]:
comp.tail()

,fips,state,name,pop2000,pop2010,pop2011,pop2012,pop2013,pop2014,pop2015,...,poverty_under_18_2019,two_plus_races_2019,unemployment_rate_2019,uninsured_2019,uninsured_65_and_older_2019,uninsured_under_19_2019,uninsured_under_6_2019,veterans_2019,white_2019,white_not_hispanic_2019
3137,56037,Wyoming,Sweetwater County,37613.0,43806,44013.0,45042.0,45145.0,44981.0,44732.0,...,NaN,2.3,5.7,11.3,0.5,9.2,13.8,8.6,93.4,79.6
3138,56039,Wyoming,Teton County,18251.0,21294,21476.0,21709.0,22326.0,22817.0,23029.0,...,NaN,0.7,0.7,12.7,0.0,10.1,5.9,5.3,89.3,81.3
3139,56041,Wyoming,Uinta County,19742.0,21118,20899.0,20999.0,20960.0,20845.0,20780.0,...,NaN,3.5,5.5,11.2,0.6,6.8,1.8,7.4,93.4,87.5
3140,56043,Wyoming,Washakie County,8289.0,8533,8460.0,8421.0,8427.0,8288.0,8296.0,...,NaN,3.8,4.1,15.0,1.5,7.0,7.8,11.9,89.7,81.9
3141,56045,Wyoming,Weston County,6644.0,7208,7141.0,7074.0,7136.0,7142.0,7181.0,...,NaN,1.3,4.0,11.8,0.0,8.6,7.1,10.3,97.4,96.4


In [39]:
df = crime.merge(comp,how='left',left_on=['fips'], right_on=['fips'])

In [40]:
df.head()

,STUDYNO,EDITION,PART,IDNO,FIPS_ST,FIPS_CTY,CPOPARST,AG_ARRST,JURFLAG,COVIND,...,poverty_under_18_2019,two_plus_races_2019,unemployment_rate_2019,uninsured_2019,uninsured_65_and_older_2019,uninsured_under_19_2019,uninsured_under_6_2019,veterans_2019,white_2019,white_not_hispanic_2019
0,This variable is no longer in use,"2nd ed, 2014","Arrests, All Ages",1,01,001,56219,3,Multiple counties,98.4525,...,23.2,2.2,3.5,7.1,0.0,1.7,1.7,12.6,76.8,74.6
1,This variable is no longer in use,"2nd ed, 2014","Arrests, All Ages",2,01,003,182265,13,One cnty,100.0000,...,13.4,1.7,4.0,8.9,0.3,3.8,2.2,11.8,86.2,83.1
2,This variable is no longer in use,"2nd ed, 2014","Arrests, All Ages",3,01,005,27457,6,One cnty,98.9839,...,50.1,1.2,9.4,11.3,0.3,3.3,3.4,6.6,46.8,45.8
3,This variable is no longer in use,"2nd ed, 2014","Arrests, All Ages",4,01,007,22866,5,Multiple counties,100.0000,...,NaN,0.6,7.0,10.7,0.0,2.0,4.5,8.0,76.8,74.5
4,This variable is no longer in use,"2nd ed, 2014","Arrests, All Ages",5,01,009,57322,6,One cnty,99.2813,...,18.4,1.6,3.1,10.8,0.2,5.9,6.1,7.7,95.5,86.9
